## W10 Sampling Based Inference part1

- 지금까지 EM을 통해 parameter를 Inference 해보았다.
- EM과 완전히 다른 방법인 Sampling Based Inference을 W10 에 배워보자.


## Basic Sampling Methods

### A. Forward Sampling

- 가장 기본적인 Sampling 방법으로 Topological order(ex Bayesian Network)에 따라 Sample을 Generate한다.

- ex) <img  src="./image/W7-8.PNG" width="45%">

- 그리고 이렇게 Sample을 Topological order에 따라 많이많이(10만개 100만개 정도) 만든 다음에 Count하면 다음과 같은 것을 구할 수 있다.
    - ex 1) $ P(E = T | MC = T ) $같은 것(Inferenc Question 2. Conditional Probability). 근데 바로 구하긴 힘드니, $ {{P(E = T , MC = T )}\over{P(MC = T)}} $를 다 세서 원하는 확률을 구할 수 있다.
    - W7에서는 variable Elimination을 사용해 구했지만, 지금 이 방법은 아주 쉬우면서 무식하고 컴퓨터가 따라하기 쉬운 방법이다.
    - 문제점: Random하게 계속 샘플하면, 정확한 값을 찾을 수 있지만 지금 방법은 그렇지 않음
    - 그러나 아주 많이 반복하면 오차가 무시할 수 있을 정도로 작아지겠지만, 시간이 오래걸린다. 그래서 현실에 쓰이긴 어렵다.
    
    - ex 2) Forward Sampling in GMM <img  src="./image/W10-1.PNG" width="90%">
    - $ \pi $를 통해 z를 샘플링하고, 평균과 분산 파라미터를 통해 x, mixture distribution을 샘플링 할 수 있다.
    - 이렇게 샘플링하여 모드(?)나 분포의 파라미터를 찾을 수 있다.


### B. Rejection Sampling

- Forward Sampling에는 Topological order에 따라 Sample을 Generate 계속한다음에 count하는 것이다.
    - ex) W7-8의 그림에서 P(E=T|MC=T,A=F)을 알고 싶을 때, 
    - Topology를 따라 중간에 우리의 조건에 안맞으면 reject하고 (ex) E=T 이후에 A = T이면 sampling멈추고 reject)
    - Count(E=T,MC=T,A=F)/# of Samples을 답한다.


- Rejection Sampling from Numerical View
    - 수치적인 관점에서의 Rejection Sampling (위에는 discrete한 경우에서이다)
    - q(x)는 Normal을 따르고, p(x)는 알아내고 싶은 mixture distribution이라고 해보자.
    - 그리고 q(x)는 p(x)를 envelope할 수 있게(감쌀 수 있게) M을 곱하자.
    - <img  src="./image/W10-2.PNG" width="80%">
    - 그리고 xi라는 지점에서 sampling되었을 때, p(x)의 파라미터는 알 수 없지만 evaluation을 할 수 있어 p(xi)의 값(높이)은 알 수 있다면, 
    - q(xi)/Mq(xi)을 이용해, 샘플링된 xi값의 y값이 q(xi)/Mq(xi)보다 크면 reject하며 그 부분은 그림으 빗금친 부분일 것이다.
    - Mq(x)가 p(x)를 envelope하지 않으면, rejection sampling이 안돌아가며 이를 위해 M이 너무 크면 Rejection Region이 커지는 문제로 시간상 문제가 생길 수 있다.


- Rejection Sampling in GMM
    - <img  src="./image/W10-3.PNG" width="70%">
    - Q를 어떻게 envelop하게 하냐에 따라 샘플링 된것이 다르다. 그냥 노말 하나로 하면 3번째 봉우리가 잘 언더샘플링 된 것을 볼 수 있다.
        - 그냥 노말인 것도 M(3인 부분)값을 좀더 높여서 하면 언더샘플링 이슈 어느정도 해결가능하지만, 그러면 Rejection이 너무 많아짐.
        - 두번째 mixture로 한 Q는 평균을 어느 정도 맞춰 준것을 볼 수 있다.
    - 여전히 현실에서 쓰기는 힘들어보인다.


### Importance Sampling

- Rejection없이 샘플링 하여 낭비를 줄이자.
- Sampling의 목적은 generating PDF가 아니라, 
    - Calculating the expectation of PDF or Calculating a certain probability에 있다.
    - 기댓값이나 특정 확률을 구하는
    

- 함수 f에 대한 기대값을 계산
    - $ E(f) = \int f(z)p(z) dz = \int f(z){{p(z)}\over{q(z)}}q(z) dz $ 
    - $ \cong {{1}\over{L}} \sum_{l=1}^{L} {{p(z^{l})}\over{q(z^{l})}} f(z^{l}) $
        - 무한하게 샘플링할 수 없으니 q(z)dz를 $ {{1}\over{L}} \sum_{l=1}^{L}  $ normalizing constant로 사용해 summation으로 표현.
        - z는 sample이고 L은 sample횟수 
        - $ {{p(z^{l})}\over{q(z^{l})}} f(z^{l}) $는 다 계산 가능하다
        - p(z)는 evaluation할 수있다 가정(위처럼) , q(z)는 우리가 sampling하기 편한 distribution
    - 이 때, $ {{1}\over{L}} \sum_{l=1}^{L} f(z^{l})$ 이거는 평균이므로,
    - $  {{p(z^{l})}\over{q(z^{l})}}  $의 역할은 가중치로 전체는 가중평균을 나타냄을 알 수 있으며 가중치를 $ r^{l} =  {{p(z^{l})}\over{q(z^{l})}}  $ 로 표현한다.
        - 만약, $ p(z)^{l} $와 $ q(z)^{l} $가 정규화가 안되있으면, 다음과 같이 정규화 할 수 있다.
        - $ E(f) \cong {{1}\over{L}} \sum_{l=1}^{L} {{p(z^{l})}\over{q(z^{l})}} f(z^{l})  = {{1}\over{L}} {{z_{q}}\over{z_{p}}}  \sum_{l=1}^{L} {{p(z^{l})}\over{q(z^{l})}} f(z^{l})  $

- example $ P(Z>1) $
    -  $ P(Z>1) = \int_{1}^{\infty} 1_{z>1}p(z) dz = \int_{1}^{\infty} 1_{z>1}{{p(z)}\over{q(z)}}q(z) dz   $
        - Z가 1보다 큰 것만 적분할 때, z
    - $ \cong {{1}\over{L}} \sum_{l=1}^{L} {{p(z^{l})}\over{q(z^{l})}} 1_{Z^{l}>1} $
        - 1보다 큰 zl에 대해, zl에 대해 evaluation한 p(zl)값을 sampling distribution에서 나온 zl로 나누어 계산할 수 있다.
    
    
- Discrete한 도메인에서의 예제
    - <img  src="./image/W10-4.PNG" width="100%">
    - 최종적으로 우리가 관심있었던 $ P(E = T| MC = T, A = F) $의 case의 importance와 전체 likely의 weight를 나누어 normalize하면 된다.
        - $ {{1}\over{L}} \sum_{l=1}^{L} {{p(z^{l})}\over{q(z^{l})}} f(z^{l}) $ 이  SumSW/ NormSW로 대응된다.
        
        
## Sampling Based Inference
        
- Rejection Sampling과 Importance Sampling은 아직까지 연구되는 주제이다.
- 하지만, ML에서 가장 많이 사용하는 Sampling기반의 inference는 Gibbs Sampling이다.
    - 그리고 깁스샘플링은 Metropolis-Hastings Algorithm의 special한 case에 속한다.
- Metropolis-Hastings Algorithm, Gibbs Sampling에 대해 알아보고 실제로 적용할 때는 어떻게 사용되는지 알아보자.

- Detour: EM Algorithm

- <img  src="./image/W8-7.PNG" width="80%">
    -  $ \sum_{Z}P(X,Z|\theta) \to ln(\sum_{Z}P(X,Z|\theta)) $에서 Z에 대해 Marginalize out 해야했기 때문에 계산하기 어려웠다.
    - 이 Sampling Based Inference가 EM알고리즘에 활용되는 부분은 Expectation의
        - ** Assign Z by $ P(Z|X,\theta) $ ** 부분이며
        - 이 부분을 이전에는 optimize하여 Z를 assign했었는데, 이를 sampling기반으로 해보자는 것이다.
    - 사실 Sampling Based Inference이 EM과 같은 GMM에 잘 적용되기보다 여러 단계의 latent variable들이 순차적으로 연결되어 있을 때 잘 적용된다.
        - ex) LDA(Latent Dirichlet Allocation): text분야의 soft clustering 에서
        - Gibbs Sampler가 Collapse되어서 아주 잘 활용될 수 있다.



### Markov Chain

- EM이 결국에는 $ \theta $를 optimize하는 것이 목적인데, 그 과정에서 Z도 나오는 것이다. 
- 여기서는 Optimization이아니라 Z를 지속적으로 sampling하다 보니 어느 순간 Optimized된 $ \theta $를 구하는 것이 다를 것이다.
    - Forward, Rejection, Importance Sampling은 i.i.d였다. (이전 샘플과 현재한 샘플이 독립)
    - 하지만, 이번에 배우는 Gibbs Sampling은 Markov Chain을 활용해 샘플링하여 이전에 버렸던 샘플도 활용해 샘플링한다.


- Markov chain
    - 노드: Each node has a ** probability distribution of states ** 확률 분포로 갖고 있다. 
        - i.e.) The probability that a state is the current state of a system
        - Concrete observation of a system: [1 0 0] $ to $ the system is at the first state
        - Stochastic observation of a system: [0.7 0.2 0.1] $ to $ the system is likely at the first state (3개의 discrete state에서 다루고 있다.)
        - The node has a vector of state probability distribution
    - 링크: Each link suggests a probabilistic state transition
        - If a system is at the first state, the probability distribution of the next state is [0.3 0.4 0.3]
        - The link has a matrix of state transition probability distribution. (Transition Matrix라고 함)

    - ex) system에 3개의 state가 있을 때 <img  src="./image/W10-5.PNG" width="80%">


- a. Accessible & Communicate
    - $ i \to j $: State j is ** Accessible ** from i if $ T_{i,j}^{k} > 0 \space and \space k \geq 0 $ 
        - state가 천이(transition이)가 가능하다.(0이 아니다.)
    - $ i \leftrightarrow j $ : State  i and j ** Communicate ** if $ i \to j $ and $ j \to i $
        - 양방향이 가능하다.

- b. Reducibility & Irreducible
    - a Markov Chain is ** Irreducible ** if $ i \leftrightarrow j, \forall i \in S, \forall j \in S $ 
        - Irreducible하면 지속적으로 이동이 가능 할 것이다.
        - [출처 Kim Hyungjun](https://medium.com/@kim_hjun/markov-chain-stationary-distribution-5198941234f6) <img  src="./image/W10-7.PNG" width="80%">
        
- c. Periodicity & aperiodic
     - State i has period d if $ d = gcd \left\{ n:T_{i,j}^{n} \right\} $
         - Greatest Common Divisor(최대공약수)
         - ex) State i에 방문하는 peroid가 4, 8, 12이면 period를 4로 보겠다.
     - If d=1, State i is ** aperiodic **
         - ex) 1, 3, 4, 11로 방문하면 aperiodic
     
- d. Transience & Recurrent
    - State j is ** Recurrent ** if $ P(inf(t>1 : X_{t} = j) < \infty | X_{0} = j) = 1$
        - state j에 특정 타임에 방문을 했는데, 나중에 방문할 확률이 분명히 등장한다면 Recurrent라 한다.
    - 그렇지 않으면 else ** transient **
         
- e. Ergodicity
    - a state is ** Ergodic ** if the state is (positive) ** recurrent ** and ** aperiodic **
        - periodic하게 언젠간 방문한다 하면 Ergodic하다.
    - Markov chain is ergodic if all states are ergodic
        - 모든 state가 Ergodic하면 Ergodic한 Markkov Chain이다 한다.
        

### Stationary Distribution

- [Linear Algebra에서 Markov Matirx와 Steady State](https://github.com/NamSahng/Summary/blob/master/LinearAlgebra%26ItsApplications_LSH/Chapter5.ipynb)

- Stationary Distribution이 Metropolis-Hastings Algorithm을 작동하게하는 핵심이다.


- Return time 먼저 정의:
    - $ RT_{i} = min \left\{ n > 0: X_{n} = i | X_{0} = i \right\} $
    - 특정 타임에서 state i를 방문했는데, 바로 다음에 방문하는 time은?
    

- Limit Theorem of Markov Chain: OR이나 산공쪽의 개념으로 완벽하게 커버는 못하지만, 필요한 것을 보자.
    - Markov Chain이 Ergodic하고 irreducible하면, 다음이 성립한다.
    - $ \pi_{i} = lim_{n \to \infty} T_{i,j}^{(n)} = {{1}\over{E\left[ RT_{i} \right]}} $
        - $ \pi_{i} $는 모든 state마다 정의 되는 그런 값인데, 특정 state에 system이 있을 확률.
        - 현재, Stationary Distribution에서 다음 transition을 해도 Stationary Distribution이 바뀌지 않음.(Steady State와 비슷해 보임)
        - $ \pi T = \pi $
    - 그리고 이 $ \pi $가 Expected Return Time의 역수와 같아진다.
    - $ \pi_{i} $는 Uniquely determined 되며 다음과 같이 정할 수 있다.
        - $ \pi_{j} = \sum_{i\in S} \pi_{i}T_{i,j} \qquad .... eq(1) $     (위에 말한 것)
        - $ \pi_{i} \geq 0 , \sum_{i\in S} \pi_{i} = 1 \qquad .... eq(2)$ (확률 분포니까 당연)


- Given $ T $에서 $ \pi $ 계산방법:
    - $ \pi(I_{|S|,|S|} -T + 1_{|S|,|S|}) = 1_{1,|S|}  $
        - 위 식이 행렬의 형태여서 이를 조금 풀고 파이를 풀면.$ \pi - \pi T + \pi = 1 $
        - 그리고 이를 나누어서 보면 위의  Limit Theorem of Markov Chain의 식들에서 온것을 알 수있다.
        - $ \pi - \pi T $ 이 부분은 eq(1)에서
        - $ \pi = 1 $ 의 부분은 eq(2)의 두 번째 것에서
    - 그래서, $ \pi = 1_{1,|S|}(I_{|S|,|S|} -T + 1_{|S|,|S|})^{-1} $ 역행렬로서 구할 수 있다.


- 2개의 Transition Matrix에 대해 pi를 구해보는 실험.

- <img  src="./image/W10-6.PNG" width="80%">
    - pi * T = T를 보임으로써 stationary distribution을 잘 구했음을 확인 할 수 있다.
    - 그리고 위에서 구한 2가지 pi들에 대해 Reversible한지 보자.
    - Reversible markov chain은 다음과 같은 Balance Equation을 만족해야 한다.
        - $ \pi_{i} T_{i,j} = \pi_{i} T_{j,i} $
        - 즉 여기서 state i에서 j로갈 확률과 j에서 i로 갈 확률이 동일한지 보면
- <img  src="./image/W10-8.PNG" width="80%">
    - state 1에서 2로 그다음 state 2에서 1로 가는 확률 계산
    - 결과는 오른쪽 것만 Reversible한 것을 알 수 있고, Sationary이면 꼭 reversible은 아니지만,
        - Reversible이면 Stationary인 것은 증명이 되어있다. (Stationary가 강한 conditon)

- 그리고 이 Balance Equation (or Detailed Balace)로 Stationary Distribution을 구하는 것을 계속 해볼 것이다.


### Markov Chain for Sampling

- Forward, Rejection, Importance Sampling의 문제점은 sample들을 i.i.d하게 뽑아 이전 sample을 사용하지 않은 것에 있다. (Importance도 결국은 Sampling Weight을 주는데만 사용했지만 i.i.d는 마찬가지)


- $ P(E|MC=T, A=F) $ 나 E가 latent일 경우와 같은 $ P(Z|MC=T, A=F) $ 와 같은 것을 Markov Chain을 이용한 sampling으로 어떻게 inference 하는 것인가. 
    - 그리고 어떻게 E나 Z에 Assign할 것인가.

- Sequence of Random Variable에 대한 Assign을 Process 형태로.
    - <img  src="./image/W10-9.PNG" width="80%">
    - z1에서 Assign된 정보를 활용해 다음 z+1에 Assign을 한다는 것이다. 이렇게 쭈루룩
    

- 기존의 Operation Research분야에서는 Markov Chain 또는 Markov Chain Analysis는 
    - Stationary Distritbution, Return time, 다음 상품은 언제오는지, 큐는 각각 어느 size일까?를 찾는데 관심이 있었다.
    - given Transition Matrix(rule), $ p(z^{t+1}| z^{t}) $ interested in finding $ \pi(z) $ Stationary Distritbution.
  
  
- ** Markov Chain Monte Carlo **
    - 우리는 어떻게 보면 반대로 target Stationary Distritbution를 알고 있다하고,
    - 그리고 이 Stationary Distritbution을 만드는 Transition Matrix(rule)는 무엇인지와 
    - 그리고 어떻게 하면 만든 Transition Matrix(rule)을 잘 설명할 수 있는지에 관심이 있다.
        - Transition Matrix(rule)을 잘 만드는 방법은 우리가 원하는 Stationary Distritbution을 표현할 수 있도록 Transition Matrix(rule)를 잘 만들어서
        - 그리고 이를 활용해 sampling을 매 term마다 해보자.
    - 이렇게 처음에는 맘대로 정한 assignment가 되겠지만 이걸 지속적으로 현재 sampling된 assignment를 참조해 다음 sampling을 해보면
    - 나중에는 결국에는 우리가 원했던 Stationary Distritbution과 유사해 질 것 같다는 것이  Markov Chain Monte Carlo 의 핵심 아이디어이다.
    - ex)<img  src="./image/W10-10.PNG" width="100%">
    - 빨강은 False, 파랑은 True
    - 노란색은 Evidence 노드들이고 초록색은 Latent Variable이라고 하면,
    - 계속 이전의 sample들을 감안(확률적으로)하여 sample할 것이다.


<br><br>
### Reference:

- 문일철 교수님, 인공지능 및 기계학습 개론 II, https://www.edwith.org/machinelearning2__17